In [35]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [36]:
data = pd.read_excel("data.xlsx")
del data["Досліди"]
data

,x0,x1,x2,x3,y1,y2
0,1,-1,-1,-1,-0.1673,-0.3723
1,1,1,-1,-1,-0.1888,-0.3460
2,1,-1,1,-1,-0.2090,-0.2090
3,1,1,1,-1,-0.3870,-0.2170
4,1,-1,-1,1,-0.2790,-0.1550
5,1,1,-1,1,-0.2875,-0.1875
6,1,-1,1,1,-0.3650,-1.2650
7,1,1,1,1,0.2890,0.2330


In [37]:
N = 8
m = 2
n = 3
y = np.array([data[f"y{i + 1}"].values for i in range(m)]).transpose()
x = np.array([data[f"x{i}"].values for i in range(n + 1)])

yu = [(1/m) * sum(y[i]) for i in range(N)]

b1 = [sum((x[i] * yu) / N) for i in range(len(x))]
b1 = [round(i, 6) for i in b1]

b2 = np.dot(np.dot(np.linalg.inv(np.dot(x, x.transpose())), x), yu)
coef = [b1, b2]
print("Коефіцієнти рівняння: ")
print(pd.DataFrame(coef, index=['b1', 'b2']))

Коефіцієнти рівняння: 
           0         1         2         3
b1 -0.257088  0.120613 -0.009162  0.004962
b2 -0.257088  0.120613 -0.009162  0.004963


In [38]:
y1 = sum([b1[i]*x[i] for i in range(n+1)])
y2 = np.dot(b2, x)
allY = [y1, y2]
print("Розрахункова змінна стану: ")
print(pd.DataFrame(allY, index=['b1', 'b2']), '\n')
y = np.array([data[f"y{i+1}"].values for i in range(m)]).transpose()

Розрахункова змінна стану: 
           0         1         2         3         4         5         6         7
b1 -0.373501 -0.132275 -0.391825 -0.150599 -0.363577 -0.122351 -0.381901 -0.140675
b2 -0.373500 -0.132275 -0.391825 -0.150600 -0.363575 -0.122350 -0.381900 -0.140675 



In [39]:
ss = []
for i in range(N):
    ss.append(
        sum([(y[i][j] - yu[i])**2 for j in range(m)])
    )

Gp = max(ss)/sum(ss)
print("Розрахунковий критерій Кохрена: ", Gp)

Розрахунковий критерій Кохрена:  0.867099508468051


In [40]:
q = 0.05
f1 = m - 1
f2 = N
print(" f1 = ", f1,"\n", "f2 = ", f2)
Gt = 0.6798

 f1 =  1 
 f2 =  8


In [41]:
dispersion = np.average(ss)
coef_dispersion = np.sqrt(dispersion/N)
print(" Дисперсія відтворюваності = ", dispersion,"\n", "Дисперсія коефіцієнтів = ", coef_dispersion)

 Дисперсія відтворюваності =  0.058384302499999985 
 Дисперсія коефіцієнтів =  0.0854285538476451


In [42]:
Tp = [abs(b1[i])/coef_dispersion for i in range(n+1)]
print(pd.DataFrame(Tp, columns=['Розрахунковий критерій Стьюдента'], index=['X0', 'X1', 'X2', 'X3']))

    Розрахунковий критерій Стьюдента
X0                          3.009392
X1                          1.411858
X2                          0.107248
X3                          0.058084


In [43]:
f = f1*f2
Tt = 2
bi = Tt * coef_dispersion

print(pd.DataFrame([
    Tp,
    (1+n)*[Tt],
    [i > Tt for i in Tp],
    [bi]*(1+n),
    [abs(i) for i in b1],
    [abs(b) > bi for b in b1]],
    columns=["X0", "X1", "X2", "X3"],
    index = ["Розрахунковий критерій Стьюдента(Tp)","Табличний критерій Стьюдента(Tt)","Перевірвка умови Tpi > Tt","Довірчий інтервал |delta Bi|", "Абсолютні значення коефіцієнта(bi)","Перевірвка умови bi > |delta Bi|"]))

                                            X0        X1        X2        X3
Розрахунковий критерій Стьюдента(Tp)  3.009392  1.411858  0.107248  0.058084
Табличний критерій Стьюдента(Tt)             2         2         2         2
Перевірвка умови Tpi > Tt                 True     False     False     False
Довірчий інтервал |delta Bi|          0.170857  0.170857  0.170857  0.170857
Абсолютні значення коефіцієнта(bi)    0.257088  0.120613  0.009162  0.004962
Перевірвка умови bi > |delta Bi|          True     False     False     False


In [44]:
f = lambda x:sum(np.array(x)*np.array(b1))
p = [f"{b1[i]}*x{i}" for i in range(1, n + 1)]
print(f"Отримана модель \ny = {b1[0]} + " + " + ".join(p))

Отримана модель 
y = -0.257088 + 0.120613*x1 + -0.009162*x2 + 0.004962*x3


In [45]:
tx = x.transpose()
ty = [f([tx[i][j] for j in range(len(tx[i]))]) for i in range(len(tx))]
adeq_dispersion = m/(N-len(b1)) * sum([(yu[i] - ty[i])**2 for i in range(N)])
Fp = adeq_dispersion/dispersion
Ft = 3.50
print(" Розрахункове значення критерію Фішера = ",  Fp,'\n', "Табличне значення критерію Фішера = ", Ft)
f1 = N - 1
f2 = N * (m - 1)
q = 0.05

 Розрахункове значення критерію Фішера =  4.016675244240522 
 Табличне значення критерію Фішера =  3.5
